In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, Input
from tensorflow.keras.models import Model, Sequential, save_model, load_model
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

from sklearn.model_selection import StratifiedKFold, KFold

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3

from keras.utils import np_utils

import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import os
import glob

import scikitplot as skplt
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

import pickle

In [12]:
fh = open('/Users/home/Documents/Respiratory Illness AI/randomness_test/recording/test2/asthma_others2_224_224_3', 'rb')
dataset = pickle.load(fh)

In [13]:
input_shape = (224, 224, 3)

n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True)
X, Y = zip(*dataset)

X_data, Y_data = np.array(X), np.array(Y)

In [14]:
def create_model(input_shape):
    baseModel = VGG19(weights=None, include_top=False, input_shape=input_shape)

    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(2, 2))(headModel)
    headModel = Flatten(name="flatten")(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(2, activation="sigmoid")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False
        
    return model 

In [15]:
def k_fold_validation(x_data, y_data, kfold, test_num, input_shape):
    index = 0
    for train_i, test_i in kfold.split(x_data,y_data):
        index += 1 
        
        x_train, x_val = x_data[train_i], x_data[test_i]
        y_train, y_val = y_data[train_i], y_data[test_i]
        
        x_train = np.array([x.reshape(input_shape) for x in x_train])
        x_val = np.array([x.reshape(input_shape) for x in x_val])
        #remember to change number of classes 
        y_train = np.array(np_utils.to_categorical(y_train, 2))
        y_val = np.array(np_utils.to_categorical(y_val, 2))

        model = create_model(input_shape)
        
        model.compile(optimizer="Adam", 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
        
        model.fit(x=x_train,
                  y=y_train,
                  epochs=25,
                  batch_size=15,
                  validation_data=(x_val, y_val))
        
        save_model(model, 
           f'asthma{test_num}_VGG19_{index}', 
           overwrite=True, 
           include_optimizer=True, 
           save_format=None, 
           signatures=None, options=None, save_traces=True)
        

In [16]:
k_fold_validation(X_data, Y_data, kfold, 2, input_shape)

Epoch 1/25
10/10 [==============================] - 38s 4s/step - loss: 0.6879 - accuracy: 0.5827 - val_loss: 0.6758 - val_accuracy: 0.7429
Epoch 2/25
10/10 [==============================] - 42s 4s/step - loss: 0.6851 - accuracy: 0.5180 - val_loss: 0.6995 - val_accuracy: 0.5143
Epoch 3/25
10/10 [==============================] - 38s 4s/step - loss: 0.7047 - accuracy: 0.5252 - val_loss: 0.6542 - val_accuracy: 0.5143
Epoch 4/25
10/10 [==============================] - 38s 4s/step - loss: 0.8525 - accuracy: 0.5180 - val_loss: 0.7108 - val_accuracy: 0.5143
Epoch 5/25
10/10 [==============================] - 39s 4s/step - loss: 0.7407 - accuracy: 0.4964 - val_loss: 0.6396 - val_accuracy: 0.6857
Epoch 6/25
10/10 [==============================] - 38s 4s/step - loss: 0.6933 - accuracy: 0.4892 - val_loss: 0.6378 - val_accuracy: 0.7429
Epoch 7/25
10/10 [==============================] - 41s 4s/step - loss: 0.7232 - accuracy: 0.5683 - val_loss: 0.6412 - val_accuracy: 0.6857
Epoch 8/25
10/10 [==

INFO:tensorflow:Assets written to: asthma2_VGG19_1/assets


INFO:tensorflow:Assets written to: asthma2_VGG19_1/assets


Epoch 1/25
10/10 [==============================] - 39s 4s/step - loss: 0.6978 - accuracy: 0.5180 - val_loss: 0.6788 - val_accuracy: 0.5143
Epoch 2/25
10/10 [==============================] - 37s 4s/step - loss: 0.7203 - accuracy: 0.5180 - val_loss: 0.7029 - val_accuracy: 0.5143
Epoch 3/25
10/10 [==============================] - 39s 4s/step - loss: 0.7419 - accuracy: 0.4964 - val_loss: 0.6748 - val_accuracy: 0.5143
Epoch 4/25
10/10 [==============================] - 38s 4s/step - loss: 0.7451 - accuracy: 0.4748 - val_loss: 0.6734 - val_accuracy: 0.5714
Epoch 5/25
10/10 [==============================] - 37s 4s/step - loss: 0.6735 - accuracy: 0.6475 - val_loss: 0.6705 - val_accuracy: 0.4857
Epoch 6/25
10/10 [==============================] - 37s 4s/step - loss: 0.7079 - accuracy: 0.5827 - val_loss: 0.6953 - val_accuracy: 0.4857
Epoch 7/25
10/10 [==============================] - 37s 4s/step - loss: 0.7061 - accuracy: 0.5540 - val_loss: 0.6746 - val_accuracy: 0.5143
Epoch 8/25
10/10 [==

INFO:tensorflow:Assets written to: asthma2_VGG19_2/assets


...

INFO:tensorflow:Assets written to: asthma2_VGG19_4/assets


INFO:tensorflow:Assets written to: asthma2_VGG19_4/assets


Epoch 1/25
10/10 [==============================] - 42s 4s/step - loss: 0.6971 - accuracy: 0.4929 - val_loss: 0.7345 - val_accuracy: 0.4412
Epoch 2/25
10/10 [==============================] - 42s 4s/step - loss: 0.7143 - accuracy: 0.5929 - val_loss: 0.7158 - val_accuracy: 0.4706
Epoch 3/25
10/10 [==============================] - 42s 4s/step - loss: 0.7159 - accuracy: 0.5429 - val_loss: 0.6874 - val_accuracy: 0.5588
Epoch 4/25
10/10 [==============================] - 41s 4s/step - loss: 0.6816 - accuracy: 0.6214 - val_loss: 0.7578 - val_accuracy: 0.4412
Epoch 5/25
10/10 [==============================] - 42s 4s/step - loss: 0.7087 - accuracy: 0.5714 - val_loss: 0.7505 - val_accuracy: 0.4706
Epoch 6/25
10/10 [==============================] - 43s 4s/step - loss: 0.7592 - accuracy: 0.5357 - val_loss: 0.6971 - val_accuracy: 0.4706
Epoch 7/25
10/10 [==============================] - 41s 4s/step - loss: 0.7232 - accuracy: 0.5500 - val_loss: 0.6883 - val_accuracy: 0.5294
Epoch 8/25
10/10 [==

INFO:tensorflow:Assets written to: asthma2_VGG19_5/assets


INFO:tensorflow:Assets written to: asthma2_VGG19_5/assets
